In [ ]:
%cd /
from google.colab import drive
drive.mount('/content/gdrive/')

!ln -s /content/gdrive/My\ Drive/ ./mydrive

%cd /content/gdrive/MyDrive/projekt_inzynierski

/
Mounted at /content/gdrive/
/content/gdrive/MyDrive/projekt_inzynierski


# Start

## Setup

In [ ]:
!pip install -q faiss-cpu sentence-transformers
!pip install sentence-transformers -q
!pip install gliner
!pip install spacy gliner transformers torch
!python -m spacy download pl_core_news_lg
!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_sm
!python -m spacy download pl_core_news_sm
!pip install langdetect

In [ ]:
import numpy as np
import pandas as pd
import torch
import faiss
import spacy
import re
from collections import defaultdict
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from gliner import GLiNER
from spacy.matcher import Matcher
from spacy.util import filter_spans
from langdetect import detect, DetectorFactory

DetectorFactory.seed = 0

# Modele
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_e5 = SentenceTransformer('intfloat/multilingual-e5-base', device=device)
model_gliner = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")
nlp_pl = spacy.load("pl_core_news_lg")
nlp_en = spacy.load("en_core_web_md")

# Dane
df_songs = pd.read_parquet("./df_full_with_embeddings.parquet")
df_tag_embeddings = pd.read_parquet("./df_unique_tag_embeddings.parquet")

In [ ]:
df_songs.head(2)

,track_id,name,artist,spotify_preview_url,tags,genre,year,duration_ms,danceability,energy,...,popularity,spotify_url,explicit,album_images,spotify_id,n_tempo,n_loudness,tags_list,tags_count,tags_embedding
0,TRIOREW128F424EAF0,Mr. Brightside,The Killers,https://p.scdn.co/mp3-preview/4d26180e6961fd46...,"rock, alternative, indie, alternative_rock, in...",None,2004,222200,0.355,0.918,...,88.0,https://open.spotify.com/track/003vvx7Niy0yvhv...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",003vvx7Niy0yvhvHt4a68B,0.619996,0.874265,"['rock', 'alternative', 'indie', 'alternative_...",6,"[-0.005010171327739954, 0.01898571476340294, -..."
1,TRRIVDJ128F429B0E8,Wonderwall,Oasis,https://p.scdn.co/mp3-preview/d012e536916c927b...,"rock, alternative, indie, pop, alternative_roc...",None,2006,258613,0.409,0.892,...,80.0,https://open.spotify.com/track/5qqabIl2vWzo9Ap...,False,"[{""height"": 640, ""width"": 640, ""url"": ""https:/...",5qqabIl2vWzo9ApSC317sa,0.730137,0.874061,"['rock', 'alternative', 'indie', 'pop', 'alter...",9,"[0.003282089252024889, 0.02081618271768093, -0..."


## Konfiguracja

In [ ]:
CONFIG = {
    # Ekstrakcja fraz
    "gliner_threshold": 0.3,
    "tag_similarity_threshold": 0.65,

    # Scoring
    "use_idf": True,
    "query_pow": 1.0,
    "audio_weight": 0.4,

    # Retrieval
    "n_candidates": 400,
    "flat_delta": 0.05,

    # Tiery jakościowe
    "min_absolute_high": 0.75,
    "min_absolute_mid": 0.50,
    "target_pool_size": 100,
    "min_required_size": 15,
    "popularity_rescue_ratio": 0.2,

    # Popularność
    "p_high": 70,
    "p_mid": 35,
    "mix": {"high": 0.40, "mid": 0.35, "low": 0.25},
    "forced_popular": 2,
    "forced_popular_min": 80,

    # Sampling
    "final_n": 15,
    "alpha": 2.0,
    "shuffle": True,
}

## Przygotowanie danych

In [ ]:
# Rozszerzenie tagów o kategorie nadrzędne
PARENT_RULES = {
    "progressive rock": "rock", "indie rock": "rock", "hard rock": "rock",
    "heavy metal": "metal", "death metal": "metal", "black metal": "metal",
    "indie pop": "pop", "synthpop": "pop",
    "drum and bass": "electronic",
}

KEYWORD_PARENTS = {
    "rock": ["rock"], "metal": ["metal"], "pop": ["pop", "britpop"],
    "hip hop": ["hip hop", "rap"], "electronic": ["electronic", "techno", "house"],
    "jazz": ["jazz"], "punk": ["punk"], "folk": ["folk"],
}

def expand_tags(tag_list):
    if not tag_list or (isinstance(tag_list, float) and pd.isna(tag_list)):
        return []
    tags = {str(t).lower().replace("_", " ").strip() for t in tag_list}

    for child, parent in PARENT_RULES.items():
        if child in tags:
            tags.add(parent)
    for parent, kws in KEYWORD_PARENTS.items():
        if any(kw in t for t in tags for kw in kws):
            tags.add(parent)
    return sorted(tags)

def _ensure_list(x):
    if isinstance(x, list): return x
    if pd.isna(x) or x is None: return []
    return [t.strip() for t in str(x).split(",") if t.strip()]

df_songs["tags_list"] = df_songs["tags"].apply(_ensure_list).apply(expand_tags)
df_songs["tag_count"] = df_songs["tags_list"].apply(len)
df_songs = df_songs.reset_index(drop=True)

# Inverted Index
def build_inverted_index(df, tags_col="tags_list"):
    inv = defaultdict(list)
    for i, tags in enumerate(df[tags_col]):
        for t in (tags or []):
            inv[t].append(i)
    return {t: np.array(v, dtype=np.int32) for t, v in inv.items()}

INV_INDEX = build_inverted_index(df_songs)
N_SONGS = len(df_songs)

# Embeddingi tagów
TAG_VECS = np.array(df_tag_embeddings["tag_embedding"].tolist(), dtype=np.float32)
TAGS = df_tag_embeddings["tag"].tolist()

# Eksperymenty

## Eksperyment 1: Bezpośrednie Wyszukiwanie Wektorowe (Naive Embedding Search)

Opis podejścia: W tym eksperymencie wektor promptu użytkownika był porównywany bezpośrednio z wektorem reprezentującym tagi utworu (kolumna tags_embedding w bazie danych). Wektor utworu powstał poprzez uśrednienie wektorów wszystkich jego tagów. Do wyszukiwania użyto indeksu FAISS (IndexFlatIP).

Dlaczego to nie zadziałało? (Wnioski):
1. Problem "Rozmycia" (Dilution Issue): Piosenki posiadające dużo tagów (np. hity mające tagi: "rock, pop, alternative, 90s, vocal, classic") miały wektor będący średnią tych wszystkich pojęć. Tracił on wyrazistość w konkretnych kierunkach.

2. Niska precyzja dla krótkich zapytań: Przy zapytaniu "rock", algorytm faworyzował niszowe utwory opisane wyłącznie tagiem "rock". Utwory popularne (z wieloma tagami) miały niższy wynik podobieństwa, mimo że były poprawne gatunkowo.

Decyzja: Metodę zastąpiono podejściem Inverted Index + Scoring, które ocenia każdy tag niezależnie.

In [ ]:
def run_experiment_naive_search(test_prompt):
    print("--- Uruchamianie Eksperymentu 1: Naive Search ---")

    if "tags_embedding" not in df_songs.columns:
        print("Błąd: Brak kolumny 'tags_embedding'. Eksperyment niemożliwy.")
        return None

    embeddings = np.array(df_songs["tags_embedding"].to_list()).astype("float32")

    d = embeddings.shape[1]
    index_naive = faiss.IndexFlatIP(d)
    index_naive.add(embeddings)

    print(f"Zbudowano indeks FAISS dla {index_naive.ntotal} utworów.")

    def search_songs_naive(prompt: str, top_k: int = 500, threshold: float = None):
        q_emb = model_e5.encode(
            [f"query: {prompt}"],
            convert_to_numpy=True,
            normalize_embeddings=True
        ).astype("float32")

        distances, indices = index_naive.search(q_emb, top_k)
        distances = distances[0]
        indices = indices[0]

        results = df_songs.iloc[indices].copy()
        results["score_naive"] = distances

        if threshold is not None:
            results = results[results["score_naive"] >= threshold]

        return results.sort_values("score_naive", ascending=False)

    print(f"\nTestowanie promptu: '{test_prompt}'")

    results = search_songs_naive(test_prompt, top_k=50, threshold=0.85)

    if not results.empty:
        display(results[['name', 'artist', 'tags_list', 'popularity', 'score_naive']].head(10))
    else:
        print("Brak wyników spełniających kryteria.")

run_experiment_naive_search("rock alternative grunge 90s")
run_experiment_naive_search("rock")

## Eksperyment 2: Ekstrakcja cech przez LLM

W tym podejściu próbowano wykorzystać generatywną sztuczną inteligencję (Large Language Model), aby "zrozumiała" intencję użytkownika i przetłumaczyła ją na ustrukturyzowane wartości cech audio (JSON).

Zastosowana metoda:

1. Użycie modelu Qwen2.5-1.5B-Instruct (relatywnie lekki model LLM).

2. Zastosowanie techniki Prompt Engineering: zdefiniowanie System Prompt, który instruuje model, jak mapować przymiotniki na wartości numeryczne Spotify (0.0 - 1.0) i wymusza format wyjściowy JSON.

Dlaczego to nie zadziałało? (Wnioski):

1. Niska jakość odwzorowania (Gubienie informacji): Model wykazuje tendencję do pomijania istotnych słów kluczowych.

    * Przykład błędu: Dla zapytania "bardzo szybka i wesoła muzyka do tańca", model zwrócił JSON: {"n_tempo": 0.9, "danceability": 0.9}, całkowicie ignorując słowo "wesoła" (brak parametru valence).

    * Pokazuje to, że mimo dużej mocy obliczeniowej, LLM bez zaawansowanego fine-tuningu jest mniej przewidywalny niż systemy regułowe.

2. Zbyt duży narzut czasowy (Latency): Generowanie odpowiedzi przez LLM trwało od kilku do kilkunastu sekund. W systemie wyszukiwania, który ma działać w czasie rzeczywistym, jest to nieakceptowalne.

3. Wymagania sprzętowe: Nawet mały model (1.5B parametrów) obciąża pamięć GPU (VRAM), co utrudnia wdrażanie systemu.

Decyzja: Zastąpiono to podejściem hybrydowym (GLiNER + Reguły), które jest tysiące razy szybsze i w pełni deterministyczne.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import json

SYSTEM_PROMPT_LLM = """You are a music feature extractor. Extract Spotify audio features from user's description.

Features (0.0 to 1.0):
- valence: 0.1=very sad, 0.9=very happy
- energy: 0.1=calm, 0.9=intense
- n_tempo: 0.1=very slow, 0.9=very fast
- danceability: 0.1=not danceable, 0.9=very danceable
- acousticness: 0.1=electronic, 0.9=acoustic
- instrumentalness: 0.1=vocals, 0.9=no vocals

Return ONLY features mentioned in description as JSON. No explanation.
Example:
User: "szybka energiczna bez słów"
Output: {"n_tempo": 0.8, "energy": 0.9, "instrumentalness": 0.9}
"""

def run_experiment_llm(user_prompt):
    print(f"--- Uruchamianie Eksperymentu 2: LLM Extraction dla '{user_prompt}' ---")

    model_name = "Qwen/Qwen2.5-1.5B-Instruct"
    print(f"Ładowanie modelu {model_name} (może to potrwać chwilę)...")

    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype="auto",
            device_map="auto"
        )
    except Exception as e:
        print(f"Błąd ładowania modelu LLM (możliwy brak pamięci VRAM): {e}")
        return

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT_LLM},
        {"role": "user", "content": user_prompt}
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    print("Generowanie odpowiedzi...")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.1,
            do_sample=True
        )

    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    print(f"Surowa odpowiedź modelu:\n{response}")

    try:
        start = response.find('{')
        end = response.rfind('}') + 1
        if start != -1 and end > start:
            json_str = response[start:end]
            features = json.loads(json_str)
            print(f"\nZinterpretowane cechy (JSON):\n{features}")
            return features
        else:
            print("Nie znaleziono JSON w odpowiedzi.")
    except json.JSONDecodeError:
        print("Błąd parsowania JSON.")

In [ ]:
run_experiment_llm("bardzo szybka i wesoła muzyka do tańca")

--- Uruchamianie Eksperymentu 2: LLM Extraction dla 'bardzo szybka i wesoła muzyka do tańca' ---
Ładowanie modelu Qwen/Qwen2.5-1.5B-Instruct (może to potrwać chwilę)...
Generowanie odpowiedzi...
Surowa odpowiedź modelu:
{"n_tempo": 0.9, "danceability": 0.9, "valence": 0.9}

Zinterpretowane cechy (JSON):
{'n_tempo': 0.9, 'danceability': 0.9, 'valence': 0.9}


{'n_tempo': 0.9, 'danceability': 0.9, 'valence': 0.9}

Rozważano wykorzystanie modelu LLM (np. Llama-3-8B lub Qwen) w roli "inteligentnego routera". Jego zadaniem miał być podział promptu na frazy i przypisanie im kategorii (Tag, Cecha Audio, Aktywność) w jednym przebiegu, zwracając gotowy JSON.

Uzasadnienie odrzucenia (Decyzja Architektoniczna): Rozwiązanie to uznano za nieoptymalne z następujących powodów:

Latency (Opóźnienie): Czekanie ~1000ms na sam parsing zapytania przez LLM drastycznie pogarsza User Experience w systemie real-time.

Niedeterministyczność: Testy wykazały (patrz: Eksperyment 2), że LLM potrafi gubić informacje (np. pominął cechę valence przy zapytaniu "wesoła muzyka") lub halucynować tagi.

Brak precyzji w ekstrakcji granic: LLM "przepisuje" tekst, czasem go zmieniając. Systemy oparte na ekstraktorach (span extraction) są bezpieczniejsze, bo pracują na oryginalnym tekście.

Wdrożone rozwiązanie alternatywne (Pipeline Hybrydowy): Zamiast jednego "magicznego" modelu LLM, zdecydowano się na zbudowanie wielostopniowego potoku przetwarzania, który zapewnia pełną kontrolę nad logiką:

Ekstrakcja (Segmentation): Wykorzystanie reguł gramatycznych spaCy Matcher do precyzyjnego wycięcia fraz (rzeczowniki, przymiotniki, związki rządu).

Wstępny Routing (Classification): Użycie lekkiego modelu GLiNER głównie do identyfikacji twardych encji (Gatunki/Tagi).

Logika Fallback & Tournament:

Wszystko, czego GLiNER nie rozpozna jako Tag, trafia do weryfikacji jako Cecha Audio lub Aktywność.

W przypadku konfliktu (np. fraza pasuje i do Aktywności, i do Audio), uruchamiany jest mechanizm turniejowy (Tournament) oparty na confidence score z wyszukiwania wektorowego (E5).

Wniosek: Złożony system regułowy wspierany lekkim modelem ML (GLiNER + E5) okazał się szybszy i bardziej przewidywalny niż jeden duży model LLM.

## Eksperyment 3: Audio Anchors (Kotwice Semantyczne) - Odrzucony

Opis podejścia: Zamiast sztywnych reguł if/else, zastosowano podejście oparte na podobieństwie semantycznym. Zdefiniowano pary zdań referencyjnych ("kotwic") opisujących skrajne wartości cech audio (np. dla cechy energy: zdanie opisujące ciszę vs. zdanie opisujące hałas).

Wektor promptu użytkownika był porównywany z wektorami obu kotwic. Wynikowa wartość cechy była obliczana na podstawie tego, do którego bieguna (0.0 czy 1.0) prompt był bardziej podobny matematycznie.

Zastsowano podejście testowe. czy jesteśmy w stanie wiele fraz wyciągnąć z 1 całego zdania. Nie jesteśmy.

Występuje tu zjawisko **"zanieczyszczenia semantycznego" (Semantic Bleed)**.
Dla modelu wektorowego słowo "taniec" jest semantycznie bliskie słowom "impreza", "radość".
W rezultacie, obecność słowa "taniec" może sztucznie zawyżyć wynik `valence` (sprawić, że piosenka zostanie oceniona jako weselsza niż jest w rzeczywistości), mimo że użytkownik wyraźnie napisał "smutna".

Wniosek był taki, że musieliśmy podzielić najpierw prompt na wiele mniejszych prostych fraz, żeby wyciągnąć poprawnie emocje. Zastosowaliśmy też potem bardziej zniuansowane podejście najpierw z wartościami (i to co jest najbliżej ma najwyższy score). To jednak też się okazało wadliwe, bo lepiej się sprawdzają przedziały (idealne wartości dla np. "szybka").

Dlaczego to nie zadziałało? (Wnioski):

1. Niejednoznaczność: Model E5 czasami "łapał" podobieństwo nie tam, gdzie trzeba. Np. słowo "Impreza" (Party) w prompcie ciągnęło w górę nie tylko danceability (co jest poprawne), ale też valence (szczęście), co wykluczało "smutne piosenki do tańca".

2. Problem "Środka Skali": Metoda ta miała tendencję do polaryzacji wyników (albo 0.1, albo 0.9), trudno było uzyskać subtelne wartości pośrednie (np. 0.3) i przewagę by miały zbyt skrajne utwory. Zrezygnowano z takiego podejścia.

3. Decyzja: Zastąpiono to podejściem, w którym GLiNER i spacy wycina konkretną frazę (np. "spokojna"), a system mapuje ją na precyzyjny zakres liczbowy.

co jeszcze próbowaliśmy?
próba wyciągnięcia fraz na wiele sposobów. Np samym glinerem,

główne problemy:
  podzielenie fraz ładnie,
  routing - tag, audio feature??

In [ ]:
AUDIO_PAIRS_SENTENCES = {
    "valence": (
        "This is suffering, crying, tragedy, depression, heartbreak, misery, death and pain.",
        "This is happiness, joy, laughter, celebration, fun, smile, excited and party."
    ),
    "energy": (
        "The music is very calm, relaxing, quiet, gentle and sleepy.",
        "The music is energetic, aggressive, intense, fast and powerful."
    ),
    "danceability": (
        "This is music for sleeping, studying, sitting and relaxing.",
        "This is music for dancing, partying, clubs and moving your body."
    )
}

def experiment_audio_anchors(prompt, model, audio_pairs, min_diff=0.035):
    """
    Oblicza wartości cech audio na podstawie bliskości semantycznej do zdań-kotwic.
    """
    print(f"--- Analiza Audio Anchors dla: '{prompt}' ---")
    preferences = {}

    prompt_vec = model.encode([f"query: {prompt} music"], normalize_embeddings=True)[0]

    for feature, (neg_text, pos_text) in audio_pairs.items():
        neg_vec = model.encode([f"query: {neg_text}"], normalize_embeddings=True)[0]
        pos_vec = model.encode([f"query: {pos_text}"], normalize_embeddings=True)[0]

        sim_neg = np.dot(prompt_vec, neg_vec)
        sim_pos = np.dot(prompt_vec, pos_vec)

        diff = sim_pos - sim_neg

        if abs(diff) < min_diff:
            continue

        score = 0.5 + (diff * 15.0)
        score = max(0.0, min(1.0, score))

        preferences[feature] = round(float(score), 2)
        print(f"Cecha: {feature:<15} | Neg: {sim_neg:.3f} | Pos: {sim_pos:.3f} | Diff: {diff:.3f} -> Wynik: {score}")

    return preferences

test_prompt_anchors = "smutna piosenka do tańca"
prefs = experiment_audio_anchors(test_prompt_anchors, model_e5, AUDIO_PAIRS_SENTENCES)
print(f"\nWywnioskowane parametry: {prefs}")

--- Analiza Audio Anchors dla: 'smutna piosenka do tańca' ---
Cecha: danceability    | Neg: 0.765 | Pos: 0.814 | Diff: 0.049 -> Wynik: 1.0

Wywnioskowane parametry: {'danceability': 1.0}


## Eksperyment 4: Ręczny podział promptu (Intelligent Split / N-grams)

Opis podejścia: Próba stworzenia własnego algorytmu do segmentacji tekstu bez użycia sieci neuronowych typu NER. Algorytm działał w oparciu o N-gramy (okno przesuwne):

1. Prompt był dzielony na wszystkie możliwe kombinacje słów (1-słowne, 2-słowne, 3-słowne...).

2. Każdy n-gram był zamieniany na wektor i porównywany z bazą cech audio.

3. Zastosowano algorytm zachłanny (Greedy Approach): wybierano te fragmenty, które miały najwyższe dopasowanie, a następnie "wykreślano" użyte słowa, aby uniknąć nakładek.

Dlaczego to nie zadziałało? (Wnioski):

1. Złożoność obliczeniowa: Generowanie i embeddowanie wszystkich n-gramów dla długiego promptu jest kosztowne.

2. Brak kontekstu gramatycznego: Algorytm "nie widzi" zależności. Fraza "nie jest smutna" mogła zostać pocięta na "nie" (ignorowane) i "smutna" (wysokie dopasowanie do valence niskiego), co całkowicie zmieniało sens.

3. Brak separacji: Matematyka wektorowa (Cosine Similarity) ma tendencję do faworyzowania dłuższych, bogatszych fraz, które "zawłaszczają" znaczenie. Długie zdanie "zjada" mniejsze, precyzyjne cechy.

spaCy Matcher: Opiera się na składni (gramatyce). Dla matchera "bardzo szybka" to ADV + ADJ, a "wesoła" to osobne ADJ. Matcher nie obchodzi znaczenie słów, tylko ich rola w zdaniu, dlatego idealnie to tnie

In [ ]:
FEATURE_DESCRIPTIONS = {
    'valence': [
        (0.1, "very low valence, very sad, melancholic, dark, gloomy emotional mood music"),
        (0.3, "low valence, bittersweet, thoughtful, introspective, moody emotional mood music"),
        (0.5, "medium valence, neutral emotional mood, neither clearly happy nor clearly sad music"),
        (0.7, "high valence, positive, pleasant, warm, cheerful, uplifting emotional mood music"),
        (0.9, "very high valence, very happy, joyful, exstatic, euphoric, bright, feel-good emotional mood music")
    ],

    'energy': [
        (0.1, "very low energy, calm, soft, gentle, ambient-like minimal intensity music"),
        (0.3, "low energy, relaxing, mellow, laid-back, smooth, low intensity music"),
        (0.5, "medium energy, balanced intensity, moderate dynamic music"),
        (0.7, "high energy, energetic, dynamic, lively, strong impact music"),
        (0.9, "very high energy, intense, aggressive, powerful, explosive music")
    ],

    'danceability': [
        (0.1, "very low danceability, not danceable, abstract or experimental, weak or irregular rhythm music"),
        (0.3, "low danceability, little groove, minimal rhythm, not primarily for dancing music"),
        (0.5, "medium danceability, some groove, steady rhythm, moderately danceable music"),
        (0.7, "high danceability, clear beat, strong groove, good for dancing, club-oriented music"),
        (0.9, "very high danceability, strong groove, infectious rhythm, perfect for dancing, party, club banger music")
    ],

    'acousticness': [
        (0.1, "very low acousticness, fully electronic, synthetic, digital, computer-generated sound music"),
        (0.3, "low acousticness, mostly electronic with some subtle organic or acoustic elements music"),
        (0.5, "medium acousticness, balanced mix of acoustic and electronic instruments, hybrid sound music"),
        (0.7, "high acousticness, mostly acoustic, organic, live instruments such as accoustic guitar or piano music"),
        (0.9, "very high acousticness, fully acoustic, unplugged, natural, organic instruments only music")
    ],

    'n_tempo': [
        (0.1, "very slow tempo, very slow pace, dragging rhythm music"),
        (0.3, "slow tempo, downtempo, slow pace, relaxed rhythm music"),
        (0.5, "medium tempo, moderate pace, walking pace music"),
        (0.7, "fast tempo, uptempo, quick pace, energetic rhythm music"),
        (0.9, "very fast tempo, rapid pace, racing rhythm, frantic speed music")
    ],

    'instrumentalness': [
        (0.1, "very low instrumentalness, strong presence of vocals and lyrics, clear singing, vocal-focused track"),
        (0.5, "medium instrumentalness, mix of vocals and instrumental sections, vocals present but not constant"),
        (0.9, "very high instrumentalness, fully instrumental track, no vocals, no singing, music without lyrics")
    ],

    'n_loudness': [
        (0.1, "very low loudness, very quiet music, soft volume, low volume level, delicate dynamics music"),
        (0.3, "low loudness, quiet music, gentle volume, reduced volume level, subtle overall loudness music"),
        (0.5, "medium loudness, normal volume, balanced loudness level, neither quiet nor loud music"),
        (0.7, "high loudness, loud music, strong volume, high loudness level, impactful sound music"),
        (0.9, "very high loudness, very loud music, extremely strong volume, very high loudness level music")
    ],

    'speechiness': [
        (0.1, "very low speechiness, purely musical track, no spoken words, fully melodic music"),
        (0.3, "low speechiness, mostly music with occasional spoken words or short background phrases"),
        (0.5, "medium speechiness, balanced mix of speech and music, frequent spoken segments, rap-like or talky structure"),
    ],

    'noise': [
        # Nouns (Generic terms)
        (None, "music song track playlist list recording audio sound genre style vibe type kind number piece"),
        # Verbs (Search intent)
        (None, "I am looking for I want I need search find play listen to give me recommend show me"),
        # Adjectives (Empty fillers)
        (None, "good very good nice great best cool amazing awesome some any kind of such a")
    ]
}

In [ ]:
print("Generowanie wektorów dla AUDIO_INDEX...")
AUDIO_INDEX = {}

for feat, descs in FEATURE_DESCRIPTIONS.items():
    passages = [f"passage: {d[1]}" for d in descs]
    embeddings = model_e5.encode(passages, normalize_embeddings=True)
    AUDIO_INDEX[feat] = embeddings

print("Gotowe! AUDIO_INDEX zawiera wektory.")

Generowanie wektorów dla AUDIO_INDEX...
Gotowe! AUDIO_INDEX zawiera wektory.


In [ ]:
def generate_ngrams(words, min_n=1, max_n=None):
    """
    Generuje wszystkie możliwe n-gramy (ciągłe fragmenty tekstu).
    Np. dla ["szybki", "rock"] -> "szybki", "rock", "szybki rock"
    """
    if max_n is None:
        max_n = len(words)

    ngrams = []
    for n in range(min_n, min(max_n + 1, len(words) + 1)):
        for i in range(len(words) - n + 1):
            text = " ".join(words[i:i+n])
            ngrams.append((i, i+n, text))
    return ngrams

def score_text_to_feature(text, feature_embeddings_dict, model):
    """Oblicza maksymalne podobieństwo tekstu do którejkolwiek cechy w słowniku."""
    query_emb = model.encode([f"query: {text}"], normalize_embeddings=True)

    best_score = -1.0
    best_feat = None

    for feature_name, emb_matrix in feature_embeddings_dict.items():
        sims = cosine_similarity(query_emb, emb_matrix)[0]
        max_sim = float(np.max(sims))

        if max_sim > best_score:
            best_score = max_sim
            best_feat = feature_name

    return best_score, best_feat

def experiment_intelligent_split(prompt, feature_embeddings_dict, model, min_confidence=0.75):
    print(f"--- Analiza N-gramowa dla: '{prompt}' ---")

    words = prompt.lower().split()
    ngrams = generate_ngrams(words, min_n=1, max_n=min(6, len(words)))

    extracted_parts = []
    used_positions = set()

    for _ in range(5):
        best_iter_score = -1
        best_iter_match = None

        for start, end, text in ngrams:
            positions = set(range(start, end))
            if positions & used_positions:
                continue

            score, feat_name = score_text_to_feature(text, feature_embeddings_dict, model)

            if score > best_iter_score:
                best_iter_score = score
                best_iter_match = (start, end, text, feat_name)

        if best_iter_score >= min_confidence and best_iter_match:
            start, end, text, feat = best_iter_match
            print(f"  Wykryto: '{text}' -> Cecha: {feat} (Score: {best_iter_score:.3f})")

            extracted_parts.append({
                'text': text,
                'feature': feat,
                'score': best_iter_score
            })
            used_positions.update(range(start, end))
        else:
            break

    extracted_parts.sort(key=lambda x: words.index(x['text'].split()[0]))
    return extracted_parts

prompt_test = "bardzo szybka i wesoła muzyka"
results = experiment_intelligent_split(prompt_test, AUDIO_INDEX, model_e5)
print("\nWynik segmentacji:", [r['text'] for r in results])

--- Analiza N-gramowa dla: 'bardzo szybka i wesoła muzyka' ---
  Wykryto: 'bardzo szybka i wesoła muzyka' -> Cecha: energy (Score: 0.859)

Wynik segmentacji: ['bardzo szybka i wesoła muzyka']


1. poczatek - proba najprosztszych rozwiazan całościowych
2. podział na frazy i przypisanie ich do audio/tag
3. osobno podzial na frazy (llm, matcher + gliner (fail), matcher (sam, ost)), a potem próba przypisania routing (tu porownanie niewiele dawalo bo tagi i features audio dzialaja srednio. w poprzednim kroku zauwazono, ze gliner slabo dzieli zdanie na frazy, ale za to swietnie rozpoznaje tagi, więc postawiono zastosować system typu wydziel tagi, potem zostaw resztę (audio features i potem dojda jeszcze czynnosci to je oddzielamy mechanizmem porownania + odrobina matchera)).
4. porownanie normalnie taga do tagow, cechy do cech itp. to dziala git.

jeszcze byly eksperymenty czy wystarczy progowanie tagow (jesli ma similarity dla jakiegos taga > niz np 0.9 to tag a jak nie to nie), ale z powodu tego, że ten model ma wysoki ten wskaźnik similarity a niską rozbieżność, to prawie wsyztsko miało bardzo wysoki score i bardzo trudno było tak ustawić, żeby idealnie wpadały niektóre.

czy nasz system nie jest nadmiernie skomplikowany?

zapytac ai czy wyglada na ai przez przesadna komplikacje kodu i unikanie prostowty typowej dla czlowieka (ai zawsze rozwkleka ten kod niemiłosiernie)

## Eksperyment 5: GLiNER jako samodzielny ekstraktor (End-to-End)

Opis podejścia: Próba wykorzystania modelu GLiNER jako jedynego narzędzia do analizy tekstu. Założono, że model ten będzie w stanie samodzielnie:

Wykryć granice fraz (gdzie zaczyna się i kończy opis).

Przypisać im odpowiednie kategorie (Gatunek, Nastrój, Tempo).

Scenariusz testowy: Uruchomienie modelu na złożonym prompcie z wieloma przymiotnikami i sprawdzenie, czy wyłapie wszystkie istotne słowa kluczowe.

Dlaczego to nie wystarczyło? (Wnioski):

Niestabilność granic (Span Boundaries): GLiNER jest modelem probabilistycznym. Czasami uznawał frazę "szybki rock" za jedną encję (Gatunek), a innym razem rozdzielał "szybki" i "rock". Brak determinizmu utrudniał dalszą logikę.

Pominięcia (Missed Entities): Model świetnie radzi sobie z rzeczownikami (gatunki), ale gorzej z luźnymi przymiotnikami (np. "fajna", "dobra", "mocna"), jeśli nie pasują idealnie do definicji etykiety.

Decyzja (Podejście Hybrydowe):

Ekstrakcja (Segmentation): Przeniesiono do spaCy Matcher. Reguły gramatyczne (np. PRZYMIOTNIK + RZECZOWNIK) są sztywne i zawsze wytną poprawny fragment tekstu.

Klasyfikacja (Routing): GLiNER został zdegradowany do roli "klasyfikatora" (Routera) – dostaje już wycięty kawałek i decyduje tylko, czy to Tag (Gatunek), czy Audio Feature.

In [ ]:
def experiment_gliner_standalone(prompt):
    print(f"--- GLiNER Standalone dla: '{prompt}' ---")

    labels = [
        "gatunek muzyczny",
        "nastrój",
        "tempo",
        "przeznaczenie (aktywność)"
    ]

    entities = model_gliner.predict_entities(prompt, labels, threshold=0.1)

    print(f"Znalezione encje:")
    found_texts = []
    for e in entities:
        print(f"  • '{e['text']}' -> {e['label']} ({e['score']:.2f})")
        found_texts.append(e['text'])

    return found_texts

prompt_gliner = "szukam bardzo szybkiej muzyki rockowej do biegania i czegoś wolnego"

print("Oczekiwanie: Wykrycie 'bardzo szybkiej', 'rockowej', 'do biegania', 'wolnego'")
detected = experiment_gliner_standalone(prompt_gliner)

expected_keywords = ["szybkiej", "rockowej", "biegania", "wolnego"]
missing = [word for word in expected_keywords if not any(word in d for d in detected)]

if missing:
    print(f"\nPorażka: Model pominął słowa: {missing}")
    print("Wniosek: GLiNER gubi przymiotniki lub łączy je w zbyt długie bloki.")
else:
    print("\nUwaga: Nawet jeśli wykrył wszystko, analiza granic (gdzie kończy się opis) jest niepewna.")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Oczekiwanie: Wykrycie 'bardzo szybkiej', 'rockowej', 'do biegania', 'wolnego'
--- GLiNER Standalone dla: 'szukam bardzo szybkiej muzyki rockowej do biegania i czegoś wolnego' ---
Znalezione encje:
  • 'szukam' -> przeznaczenie (aktywność) (0.22)
  • 'bardzo szybkiej' -> tempo (0.20)
  • 'muzyki rockowej' -> gatunek muzyczny (0.80)
  • 'biegania' -> przeznaczenie (aktywność) (0.47)
  • 'wolnego' -> przeznaczenie (aktywność) (0.41)

Uwaga: Nawet jeśli wykrył wszystko, analiza granic (gdzie kończy się opis) jest niepewna.
